# Registration tools

This notebook is part of [blabla citation papier].
Before executing it, please make sure you have installed the tools properly (see installation part).

Please run the cells one by one, provide input when it is asked and click enter to validate your input, but do not modify the content of the cells.

The data structure required is the following : one main folder with the raw movies, which are 3D stacks in time. The multichannel images should be split before. The name should follow 'name_index_channel.tif' avec 'name' the common part between all of your samples, 'index' the index of your sample in the dataset and 'channel' the name of the channel.

Exemple : If my acquisition includes 3 samples with 2 channels each, dapi and ecad, i will have in my folder the following files :

________________________________________

'sample_1_dapi.tif'

'sample_1_ecad.tif'

'sample_2_dapi.tif'

'sample_2_ecad.tif'

'sample_3_dapi.tif'

'sample_3_ecad.tif'

And each file is of dimension (T,Z,X,Y)

1- Importing packages (no input required)

In [1]:
import tifffile
import os
import registrationtools
import json
import numpy as np

C:\Users\gros\Anaconda3\envs\registration-env\lib\site-packages\IO\IO.py:31: UserWarning: KLB library is not installed
  warnings.warn("KLB library is not installed")


pyklb library not found, klb files will not be generated


2) Please fill below the path to your data (ex : C:\Users\username\Desktop\DATA\20230101\) **with a slash at the end**

In [ ]:
path_to_data = (input('Absolute path to your data folder:')).replace("\\", "/")

3) Also fill the path where you want to save the json file of your registration **with name.json file** (this might be useful in case of debugging or safety checks)
example : C:\Users\username\Desktop\CODES\Registration\json_files\temporal.json

In [3]:
path_to_json = (input('Absolute path to save the json file :')).replace("\\", "/")

Absolute path to save the json file :C:\Users\gros\Desktop\CODES\Registration\json_files\essai.tif


4) To find some parts of the code, the tools need to know where in your installation is your bin folder located. If you followed the procedure from the paper, the path should look like should look like C:\Users\username\Anaconda3/envs\registration-env\Library\bin\

In [4]:
path_to_bin = (input('Absolute path to the bin folder :')).replace("\\", "/")

Absolute path to the bin folder :C:\Users\gros\Anaconda3/envs\registration-env\Library\bin\


5. Fill here the name of your files without the number of the sample and the channel. For example, if I have 'sample_1_dapi.tif','sample_1_ecad.tif','sample_1_dapi.tif', here the common name is 'sample".

In [3]:
name = input('Common name of your movies (without any extention) :')

Common name of your movies (without any extention) :FGF_100h


6. Give here the number of samples that you want to register. We suppose that these samples are acquired with the same channels, have the same name patterns and acquired in the same configuration.
Then, give the different numbers that are going to be registered (could be 1, 2 and 3, or 1, 3, 7, no need to be ordered or adjacent)

In [ ]:
number_samples = int(input('Number of samples :'))

In [ ]:
samples=[]
for i in range(number_samples) :
    s = int(input('Index of sample n°'+str(i+1) +':'))
    samples.append(s)
print('samples =',samples)

7. Give here the number of channels of the acquisition, and their names.

In [6]:
number_channels = int(input('Number of channels :'))

Number of channels :2


8. If the image has multiple channels, you will need one channel who will be the reference to register the other ones. The most 'stable' one, expressed everywhere, is generally a good option.

In [8]:
if number_channels == 1:
    print('There is only one channel')
else :  
    ch_ref = input('reference channel :')

reference channel :sulfo


In [9]:
channels=[]
if number_channels == 1:
    ch = input('Name of the channel : ')
    channels.append(ch)
    ch_ref=ch
else : 
    print('Name of channel n°1 :', ch_ref)
    channels.append(ch_ref)
    for n in range(1,number_channels) :
        ch = input('Name of channel n°'+str(n+1) +' : ')
        channels.append(ch)

Name of channel n°1 : sulfo
Name of channel n°2 : h2b


9. Please also give the voxel size of your image, for example [0.2,0.2,1]. This can be found in Fiji, Image>Show Info

In [12]:
print('Voxel size of your original image (XYZ) :')
x= float(input('X :'))
y= float(input('Y :'))
z= float(input('Z :'))
ref_voxel = [x,y,z]
print('initial voxel size',ref_voxel)

Voxel size of your original image (XYZ) :
X :0.62
Y :0.62
Z :2
initial voxel size [0.62, 0.62, 2.0]


10. If for any reason you prefer having an other voxel size for the registered image (for example isotropic, meaning the same voxel size in every dimension), you can tune it below. You can also keep the same size.

In [13]:
change_voxel_size = int(input('Do you want to change the voxel size of your movies ? (1 for yes, 0 for no) : '))
if change_voxel_size==1 :
    print('\nVoxel size of your image after transformation (XYZ) (ex :[1,1,1]) :')
    x= float(input('X :'))
    y= float(input('Y :'))
    z= float(input('Z :'))
    voxel_size_output = [x,y,z]
elif change_voxel_size==0 :
    voxel_size_output = ref_voxel
print('\nVoxel size after transformation',voxel_size_output)

Do you want to change the voxel size of your movies ? (1 for yes, 0 for no) : 0

Voxel size after transformation [0.62, 0.62, 2.0]


In [14]:
channels

['sulfo', 'h2b']

11. The registration happens below. If there is an error, it will be printed in the terminal window. To solve it, please consider the 'Troubleshooting' section.

In [15]:
for i in samples :
    for c in channels:
        print(i,c)
        movie = tifffile.imread(rf'{path_to_data}{name}_{i}_{c}.tif') ##if you want to change the naming of your files
        directory = str(i)+"_"+c
        #creates data structure : one directory per channel and per sample
        os.mkdir(os.path.join(path_to_data,directory))
        os.mkdir(os.path.join(path_to_data+directory,"trsf"))
        os.mkdir(os.path.join(path_to_data+directory,"proj"))
        os.mkdir(os.path.join(path_to_data+directory,"stackseq"))
        ## from movie to image sequence
        for t in range(movie.shape[0]) :
            stack = movie[t,:,:,:]
            tifffile.imwrite(path_to_data+directory+"/stackseq/movie_t"+str(format(t,'03d')+'.tif'),stack)

1 sulfo



KeyboardInterrupt



In [21]:
for i in samples :
    for c in channels:
        directory = str(i)+"_"+c
        
        movie = tifffile.imread(rf'{path_to_data}{name}_{i}_{c}.tif') ##if you want to change the naming of your files
        ref_timepoint = int(movie.shape[0]/2)
        if c==ch_ref:
            compute_transf = 1
        else :
            compute_transf = 0

        # # # # ##JSON FILE CONFIG
        data = {
          "path_to_data": path_to_data+directory+"/stackseq/",
          "file_name": "movie_t{t:03d}.tif",
          "trsf_folder": rf'{path_to_data}{str(i)}_{ch_ref}/trsf/',
          "output_format": path_to_data+directory+"/proj/",
          "projection_path": path_to_data+directory+"/proj/",
          "check_TP": 0,
          "voxel_size": ref_voxel,
          "voxel_size_out" : voxel_size_output,
          "first": 0,
          "last": movie.shape[0]-1,
          "not_to_do": [],
          "compute_trsf": compute_transf,
          "ref_TP": ref_timepoint,
          "trsf_type": "rigid",
          "padding": 1,
          "recompute": 1,

          "apply_trsf":1,
          "out_bdv": "",
          "plot_trsf":0

        }

        json_string=json.dumps(data)
        with open(path_to_json,'w') as outfile :
            outfile.write(json_string)

        tr = registrationtools.TimeRegistration(data)
        tr.run_trsf()


Starting experiment
The registration will run with the following arguments:

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- File format -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
path_to_data             : C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_sulfo/stackseq/
file_name                : movie_t{t:03d}.tif
trsf_folder              : C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_sulfo/trsf/
output_format            : C:/Use

<tifffile.TiffFile 'movie_t096.tif'> ImageJ series metadata invalid or corrupted file


Failure of {'path_to_data': 'C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_sulfo/stackseq/', 'file_name': 'movie_t{t:03d}.tif', 'trsf_folder': 'C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_sulfo/trsf/', 'output_format': 'C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_sulfo/proj/', 'projection_path': 'C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_sulfo/proj/', 'check_TP': 0, 'voxel_size': [0.62, 0.62, 2.0], 'voxel_size_out': [0.62, 0.62, 2.0], 'first': 0, 'last': 199, 'not_to_do': [], 'compute_trsf': 1, 'ref_TP': 100, 'trsf_type': 'rigid', 'padding': 1, 'recompute': 1, 'apply_trsf': 1, 'out_bdv': '', 'plot_trsf': 0}
failed to read 183280384 bytes, got 2548

Starting experiment
The registration will run with the following arguments:

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<tifffile.TiffFile 'movie_t000.tif'> ImageJ series metadata invalid or corrupted file


applyTrsf failde, retrying now


<tifffile.TiffFile 'movie_t000.tif'> ImageJ series metadata invalid or corrupted file


Failure of {'path_to_data': 'C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_h2b/stackseq/', 'file_name': 'movie_t{t:03d}.tif', 'trsf_folder': 'C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_sulfo/trsf/', 'output_format': 'C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_h2b/proj/', 'projection_path': 'C:/Users/gros/Desktop/DATA/20230207_fgf/20230208/2_h2b/proj/', 'check_TP': 0, 'voxel_size': [0.62, 0.62, 2.0], 'voxel_size_out': [0.62, 0.62, 2.0], 'first': 0, 'last': 199, 'not_to_do': [], 'compute_trsf': 0, 'ref_TP': 100, 'trsf_type': 'rigid', 'padding': 1, 'recompute': 1, 'apply_trsf': 1, 'out_bdv': '', 'plot_trsf': 0}
failed to read 183280384 bytes, got 2548


12. The previous cell saved an image sequence. Here is some code to save the output as a movie.

In [12]:
for i in samples :
    for c in channels:
        directory = str(i)+"_"+c
        stack0 =tifffile.imread(rf"{path_to_data}/{directory}/proj/movie_t000.tif")
        movie = tifffile.imread(rf'{path_to_data}{name}_{i}_{c}.tif') ##if you want to change the naming of your files
        registered_movie = np.zeros((movie.shape[0],stack0.shape[0],stack0.shape[1],stack0.shape[2]),dtype=np.float16) #one movie per channel, of format (t,z,y,x).Datatype uint16 or float34 is necessary to export as hyperstack
        for t in range(movie.shape[0]) :
            stack =tifffile.imread(rf"{path_to_data}/{directory}/proj/movie_t{format(t,'03d')}.tif")
            #we take each stack in a given timepoint
            registered_movie[t,:,:,:]=stack #and put it in a new hyperstack
        tifffile.imwrite(path_to_data+rf"/registermovie_{c}.tif",registered_movie.astype(np.float16)) #write a hyperstack in the main folder
        print(registered_movie.shape)

(200, 164, 787, 794)
(200, 164, 787, 794)


Input:
- 1 file/channel containing all timepoints
- 1 file containgin all channels and timepoints -> split channels and times in 1 folder/channel and 1 stack/ timepoint
- 1 file/channel/timepoint -> nothing to do

Output: 1 folder with all time points and all channels separated in individual stacks


In [30]:
from IO import imread, imsave
# p = f"/path/to/data/name_t{t:04d}.tif"
def combined_stack_to_sequence(
    im_path: str,
    nb_timepoints: int,
    nb_channels:int = 1
):
    im = imread(im_path)
    if 1<nb_channels:
        channel_pos = np.where(im.shape==nb_channels)[0]
    time_pos = np.where(im.shape==nb_timepoints)[0]
    